https://www.kaggle.com/code/ducanger/wids-2023-catboost-regression

In [11]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/Shareddrives/Kaggle/Kaggle/Scripts/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Setup

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import FunctionTransformer
import lightgbm as lgbm


## Load Dataset

In [4]:
train_raw = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Output/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'
train_raw

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

## Preprocessing

In [8]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    return train, test

def fill_na(df):
    dataknn = df.copy()
    missing_cols = dataknn.columns[dataknn.isnull().any()].tolist()
    # First, use KNN imputation to fill in missing values in the first column
    imputer1 = KNNImputer(n_neighbors=5)
    dataknn[missing_cols[0]] = imputer1.fit_transform(dataknn[[missing_cols[0]]])

    # Then, use iterative imputation to fill in missing values in the second and third columns
    imputer2 = IterativeImputer()
    for col in missing_cols[1:]:
        dataknn[col] = imputer2.fit_transform(dataknn[[col]])
    return dataknn

def categorical_encode(train, test, cat_cols):
    le = LabelEncoder()
    for col in cat_cols:
        train[col] = le.fit_transform(train[[col]])
        test[col] = le.transform(test[[col]])
    return train, test

def one_hot(train, test):
  one_hottrain = pd.get_dummies(train['climateregions__climateregion'])
  one_hottest = pd.get_dummies(test['climateregions__climateregion'])
  features1 = one_hottrain.columns
  features2 = one_hottest.columns
  train[features1] = one_hottrain[features1]
  test[features2] = one_hottest[features2]   
  train = train.drop(['climateregions__climateregion','index'],axis = 1, inplace=False)
  test = test.drop(['climateregions__climateregion','index'],axis = 1, inplace=False)   
  return train, test

def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day'] = df['startdate'].dt.day
    return df

def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

def add_lag(train, test):
  train["lag_a"]=train["contest-wind-h500-14d__wind-hgt-500"].shift(1)
  train["lag_b"]=train["contest-prwtr-eatm-14d__prwtr"].shift(1)
  train["lag_c"]=train["contest-pevpr-sfc-gauss-14d__pevpr"].shift(1)
  test["lag_a"]=test["contest-wind-h500-14d__wind-hgt-500"].shift(1)
  test["lag_b"]=test["contest-prwtr-eatm-14d__prwtr"].shift(1)
  test["lag_c"]=test["contest-pevpr-sfc-gauss-14d__pevpr"].shift(1)
  return train, test

def feature_scale(train, test):
    scaler = MinMaxScaler()
    train_ = scaler.fit_transform(train)
    test_ = scaler.transform(test)
    train = pd.DataFrame(train_, columns = train.columns)
    test = pd.DataFrame(test_, columns = test.columns)
    return train, test

def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    #train, test = categorical_encode(train, test, cat_cols=['climateregions__climateregion'])
    #train, test = one_hot(train, test)
    #train, test = add_lag(train, test)
    
    to_drop = identify_correlated(train.copy(), threshold=.85)
    drop_cols = ['index', 'startdate', 'climateregions__climateregion', target]
    drop_cols.extend(to_drop)
    features = [col for col in train.columns if col not in drop_cols]
    
    X = train[features]
    y = train[target]
    X_test = test[features]
    X, X_test = feature_scale(X, X_test)
    
    return X, y, X_test

## Train and Validation

In [9]:
# Bsk Dfb
train1=train_raw.loc[train_raw['climateregions__climateregion'] == 'BSk']
test1=test_raw.loc[test_raw['climateregions__climateregion'] == 'BSk']
X, y, X_test = feature_engineering(train1, test1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
print(f'Train_shape: {X_train.shape}    |   Val_shape: {X_val.shape}    |   Test_shape: {X_test.shape}')

Train_shape: (93546, 173)    |   Val_shape: (46075, 173)    |   Test_shape: (11651, 173)


In [10]:
test_index=test1["index"]

### CatBoost

In [13]:

cb_params = {'iterations': 10000,
            'learning_rate': 0.01,
            'depth': 6,
            'l2_leaf_reg': 3,
            'bagging_temperature': 0,
            'border_count': 128,
            'loss_function': 'RMSE',
            'random_seed': 42,
            'task_type': 'GPU',
            'verbose': 500}

model = CatBoostRegressor(**cb_params)
model.fit(X_train, y_train,  eval_set=(X_val, y_val))

0:	learn: 9.3107446	test: 9.3407598	best: 9.3407598 (0)	total: 4.59ms	remaining: 45.9s
500:	learn: 1.3781530	test: 1.3937974	best: 1.3937974 (500)	total: 1.68s	remaining: 31.8s
1000:	learn: 1.0656971	test: 1.0825734	best: 1.0825734 (1000)	total: 3.37s	remaining: 30.3s
1500:	learn: 0.9067816	test: 0.9259590	best: 0.9259590 (1500)	total: 5.04s	remaining: 28.5s
2000:	learn: 0.7993953	test: 0.8207477	best: 0.8207477 (2000)	total: 6.72s	remaining: 26.9s
2500:	learn: 0.7228302	test: 0.7462604	best: 0.7462604 (2500)	total: 8.39s	remaining: 25.2s
3000:	learn: 0.6644284	test: 0.6889921	best: 0.6889921 (3000)	total: 10s	remaining: 23.4s
3500:	learn: 0.6168704	test: 0.6424093	best: 0.6424093 (3500)	total: 11.7s	remaining: 21.7s
4000:	learn: 0.5786131	test: 0.6052542	best: 0.6052542 (4000)	total: 13.3s	remaining: 20s
4500:	learn: 0.5466275	test: 0.5743378	best: 0.5743378 (4500)	total: 15s	remaining: 18.4s
5000:	learn: 0.5182861	test: 0.5470656	best: 0.5470656 (5000)	total: 16.7s	remaining: 16.7s
5

In [21]:
pred=model.predict(X_test)
preds = pd.DataFrame(pred, columns=[target], index=test_index.index)
preds['index']=test1['index']

with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/pred_v61.csv', 'w', encoding = 'utf-8-sig') as f:
    preds.to_csv(f,index=False)

In [19]:
test1['index']

610      376344
611      376345
612      376346
613      376347
614      376348
          ...  
30861    406595
30862    406596
30863    406597
30864    406598
30865    406599
Name: index, Length: 11651, dtype: int64

In [ ]:
model.get_feature_importance()
# 1,12,17

array([4.61095092e+00, 5.53277756e-01, 2.48695953e-01, 2.18241636e-01,
       1.94529505e-01, 5.84033096e+00, 2.93188721e-01, 7.21154752e-02,
       2.93658082e-01, 2.33947452e+00, 2.10472114e-01, 1.24356717e-01,
       3.78685744e-01, 4.17207878e-01, 2.50944433e-01, 6.54424492e+00,
       5.99002518e-03, 6.96954856e-03, 5.67367523e-03, 1.44476186e-02,
       8.16393967e-03, 1.09172442e-02, 1.08696484e-02, 1.39346414e-02,
       1.20869282e-02, 8.22960767e-02, 1.29596171e-02, 2.29607353e-02,
       1.43861885e-02, 6.69518721e-03, 1.80425946e-02, 6.79369424e-03,
       4.48458489e-03, 4.12698153e-02, 6.04863062e-03, 3.93920647e-02,
       1.23223781e-01, 3.10898957e-01, 1.47747862e-02, 1.32244605e-02,
       1.85471501e-02, 8.26392901e-03, 1.07614645e-02, 7.22198894e-03,
       1.26790746e-02, 1.35376474e-02, 4.64452024e-03, 1.60219502e-02,
       2.91592561e-02, 1.31434933e-01, 3.76702527e-03, 6.67884781e-03,
       5.69030781e-03, 8.29493762e-03, 1.51782565e-03, 2.27912754e-03,
      

In [ ]:
X_train.columns[17] # contest-wind-h500-14d__wind-hgt-500
X_train.columns[12] # contest-prwtr-eatm-14d__prwtr
X_train.columns[1] # contest-pevpr-sfc-gauss-14d__pevpr

'contest-pevpr-sfc-gauss-14d__pevpr'

### LightGBM

In [ ]:
lightGBM_params = {
          
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'max_depth': 8,
          'num_leaves': 16,
          'learning_rate': 0.05,
          'feature_fraction': 0.5,
          'bagging_fraction': 0.9,
          'bagging_freq': 5,
          'early_stopping_round': 50,
          'n_estimators': 15000

}


reg_lgb = lgbm.LGBMRegressor(**lightGBM_params)

reg_lgb.fit(X_train, y_train, eval_set=(X_val, y_val),verbose=100)



Training until validation scores don't improve for 50 rounds.
[100]	valid_0's rmse: 1.54186
[200]	valid_0's rmse: 1.24907
[300]	valid_0's rmse: 1.10351
[400]	valid_0's rmse: 1.00629
[500]	valid_0's rmse: 0.939298
[600]	valid_0's rmse: 0.889146
[700]	valid_0's rmse: 0.848793
[800]	valid_0's rmse: 0.816949
[900]	valid_0's rmse: 0.780708
[1000]	valid_0's rmse: 0.752863
[1100]	valid_0's rmse: 0.729629
[1200]	valid_0's rmse: 0.70807
[1300]	valid_0's rmse: 0.690705
[1400]	valid_0's rmse: 0.674575
[1500]	valid_0's rmse: 0.660125
[1600]	valid_0's rmse: 0.646802
[1700]	valid_0's rmse: 0.633042
[1800]	valid_0's rmse: 0.618451
[1900]	valid_0's rmse: 0.60714
[2000]	valid_0's rmse: 0.597864
[2100]	valid_0's rmse: 0.586239
[2200]	valid_0's rmse: 0.577036
[2300]	valid_0's rmse: 0.565868
[2400]	valid_0's rmse: 0.556591
[2500]	valid_0's rmse: 0.547813
[2600]	valid_0's rmse: 0.539601
[2700]	valid_0's rmse: 0.532266
[2800]	valid_0's rmse: 0.524218
[2900]	valid_0's rmse: 0.51658
[3000]	valid_0's rmse: 0.5

LGBMRegressor(bagging_fraction=0.9, bagging_freq=5, early_stopping_round=50,
              feature_fraction=0.5, learning_rate=0.05, max_depth=8,
              metric='rmse', n_estimators=15000, num_leaves=16,
              objective='regression')

In [ ]:
submit[target] = reg_lgb.predict(X_test)
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v38.csv', 'w', encoding = 'utf-8-sig') as f:
    submit.to_csv(f,index=False)

### XGBoost

In [ ]:
import xgboost as xgb
reg_xgb = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=15000,
                           objective='reg:linear',
                           max_depth=4,
                           early_stopping_rounds=100,
                           learning_rate=0.01)


reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),),verbose=100)

submit[target] = reg_xgb.predict(X_test)
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v40.csv', 'w', encoding = 'utf-8-sig') as f:
    submit.to_csv(f,index=False)

[02:58:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:14.8885
[100]	validation_0-rmse:6.07583
[200]	validation_0-rmse:3.04979
[300]	validation_0-rmse:2.08794
[400]	validation_0-rmse:1.7629
[500]	validation_0-rmse:1.62778
[600]	validation_0-rmse:1.55144
[700]	validation_0-rmse:1.4963
[800]	validation_0-rmse:1.45148
[900]	validation_0-rmse:1.41185
[1000]	validation_0-rmse:1.37667
[1100]	validation_0-rmse:1.3463
[1200]	validation_0-rmse:1.31748
[1300]	validation_0-rmse:1.29322
[1400]	validation_0-rmse:1.2694
[1500]	validation_0-rmse:1.24775
[1600]	validation_0-rmse:1.22718
[1700]	validation_0-rmse:1.20413
[1800]	validation_0-rmse:1.18333
[1900]	validation_0-rmse:1.16458
[2000]	validation_0-rmse:1.14724
[2100]	validation_0-rmse:1.13139
[2200]	validation_0-rmse:1.11391
[2300]	validation_0-rmse:1.09832
[2400]	validation_0-rmse:1.08302
[2500]	validation_0-rmse:1.06554
[2600]	validation_0-rmse:1.05
[

In [ ]:
import xgboost as xgb
reg_xgb = xgb.XGBRegressor(base_score=0.8, 
                           n_estimators=15000,
                           objective='reg:linear',
                           max_depth=4,
                           early_stopping_rounds=100,
                           learning_rate=0.01)


reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),),verbose=100)

submit[target] = reg_xgb.predict(X_test)
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v41.csv', 'w', encoding = 'utf-8-sig') as f:
    submit.to_csv(f,index=False)

[11:29:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:14.6658
[100]	validation_0-rmse:6.0027
[200]	validation_0-rmse:3.03067
[300]	validation_0-rmse:2.08471
[400]	validation_0-rmse:1.76402
[500]	validation_0-rmse:1.62942
[600]	validation_0-rmse:1.55119
[700]	validation_0-rmse:1.49461
[800]	validation_0-rmse:1.45112


In [ ]:
import xgboost as xgb
reg_xgb = xgb.XGBRegressor(base_score=0.8, 
                           n_estimators=15000,
                           objective='reg:linear',
                           max_depth=6,
                           early_stopping_rounds=100,
                           learning_rate=0.01)


reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),),verbose=100)

submit[target] = reg_xgb.predict(X_test)
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v42.csv', 'w', encoding = 'utf-8-sig') as f:
    submit.to_csv(f,index=False)

In [ ]:
import xgboost as xgb
reg_xgb = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=20000,
                           objective='reg:linear',
                           max_depth=4,
                           early_stopping_rounds=100,
                           learning_rate=0.01)


reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),),verbose=100)

submit[target] = reg_xgb.predict(X_test)
with open('/content/drive/Shareddrives/Kaggle/Kaggle/Output/solution_v43.csv', 'w', encoding = 'utf-8-sig') as f:
    submit.to_csv(f,index=False)

## Train and Validate on KNN Interpolate

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/trainknn_all.csv')
test = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/test_all.csv')

In [ ]:
X = train.drop([target,'Unnamed: 0'],axis = 1, inplace=False)
y = train[target]
X_test = test[X.columns]

In [ ]:
to_drop = identify_correlated(X,threshold=.85)
features = [col for col in X.columns if col not in to_drop]
X = train[features]
X_test = test[features]
X, X_test = feature_scale(X, X_test)